  <div id="Z_analysis" style="float:left; clear:both; padding: 0.0cm 0.0cm 0.0cm 0.0cm; text-align: center; width:92%; margin:0px auto; background-color:#d1ecfc; border:none!important;">
    <table width="100%" border="0" style="float:center; clear:both; border:none!important;">
      <tr>
        <td style="width:50%; text-align: left; padding: 1cm 0.5cm 1cm 0.5cm;">
          <h1 style="font-family:verdana; font-size:180%;"><CENTER> Análisis del Bosón Z </CENTER>
          <img class="img-responsive" src="http://atlas.physicsmasterclasses.org/zpath_files/img/highslide/feynman/Z_ElectronPositron.png" alt="Visualisations" style="width:90%">
        </td>
        <td style="width:50%; padding: 0.2cm 0.2cm 0.2cm 0.2cm;">
            <img class="img-responsive" src="http://opendata.atlas.cern/DataAndTools/pictures/handmade_ZAnalysis.png" alt="Visualisations" style="width:100%">
        </td>
      </tr>
    </table>
  </div> 

<CENTER> Este notebook esta hecho con propósitos educacionales y esta escrito en C++ 

In [ ]:
#include <iostream>
#include <string>
#include <stdio.h>


<b> Paso uno: </b> Se crea una TChain, esta es una clase que es usanda para unir diferentes archivos en una sola cadena, a la cual se le van añadir en secuencia distintos archivos de ROOT que se van a usar en el análisis. Primero vamos a crear una cadena, en este caso llamada MonteCarlo, y en la celda de abajo se van a poner los eslabones de la cadena, los cuales van a seguir el orden en que esten puestos.


In [ ]:
TChain *MonteCarlo = new TChain("mini");

<b> Paso dos: </b> Se colectan las muestras que se van a utilizar en este análisis en un solo archivo llamado dataset, el se definió en el paso anterior con la clase de TChain.


Estamos usando los Datos Reales y los Monte Carlos que provee el website ATLAS OpenData para hacer el análisis.

Se puede usar los datos en local o tomarlo directamente de la página web de ATLAS OpenData, tal como se muestra a conticuación, dependiendo si se esta en linea, sin una conexión a internet o si se quiere agilizar el proceso.


In [ ]:
MonteCarlo->Add("/media/sf_ATLAS_OpenData/MC/mc_147770.Zee.root");

//De esta forma se lee directamente de Internet. Si se esta fuera de linea, o se desea mayor rapidez usar la linea anterior
//MonteCarlo->Add("http://opendata.atlas.cern/release/samples/MC/mc_147770.Zee.root");

/*Todos los Monte Carlo disponibles para el decaimiento del boson Z en un par de leptones
MonteCarlo->Add("/media/sf_ATLAS_OpenDataMC/mc_147770.Zee.root");
MonteCarlo->Add("/media/sf_ATLAS_OpenData/MC/mc_147771.Zmumu.root");
MonteCarlo->Add("/media/sf_ATLAS_OpenData/MC/mc_147772.Ztautau.root");


*/

<b> Paso tres y cuatro: </b> Primero hay que declarar cuales y que tipo de variables, de todas las contenidas en los Monte Carlo y Datos, reales van a ser utilizadas. Despues se muestra como las variables que ya se declararon anteriormente van a ser llamadas.  

In [ ]:
const int vs = 5;

Int_t   MonteCarlo_lepton_n = -1,
        MonteCarlo_runNumber = -1,  
        MonteCarlo_lepton_charge[vs], 
        MonteCarlo_lepton_type[vs];

Float_t MonteCarlo_lepton_pt[vs],
        MonteCarlo_lepton_eta[vs],
        MonteCarlo_lepton_phi[vs],
        MonteCarlo_lepton_E[vs];
        
Float_t MonteCarlo_met_et = -1,
        MonteCarlo_met_phi = -1;


In [ ]:
MonteCarlo->SetBranchAddress("lep_n",      &MonteCarlo_lepton_n);
MonteCarlo->SetBranchAddress("channelNumber",  &MonteCarlo_runNumber);
MonteCarlo->SetBranchAddress("lep_charge", &MonteCarlo_lepton_charge);
MonteCarlo->SetBranchAddress("lep_type",   &MonteCarlo_lepton_type);
MonteCarlo->SetBranchAddress("lep_pt",     &MonteCarlo_lepton_pt);
MonteCarlo->SetBranchAddress("lep_eta",    &MonteCarlo_lepton_eta);
MonteCarlo->SetBranchAddress("lep_phi",    &MonteCarlo_lepton_phi);
MonteCarlo->SetBranchAddress("lep_E",   &MonteCarlo_lepton_E);
MonteCarlo->SetBranchAddress("met_et",   &MonteCarlo_met_et);
MonteCarlo->SetBranchAddress("met_phi",   &MonteCarlo_met_phi);


<b> Paso cinco: </b> Vamos a definir un histograma en 1D. De forma general se escribe como:

TH1F <font color="purple"> <b>* </b> </font> nombre_histograma <font color="purple"> <b> = </b> </font>  <font color="green">  <b> new </b> </font>   TH1F("<font color="red"> nombre_histograma  </font>", "<font color="red"> Titulo del Histograma </font>; Nombre eje x ; Nombre eje y" ; <font color="green">  beam </font>, <font color="green">  $x_i$  </font> , <font color="green">  $x_f$  </font>);</p>

In [ ]:
TH1F *h_Inv_Mass = new TH1F("h_Inv_Mass","Masa Invariante Candidato a Z; Masa Invariante [GeV]; eventos",20,60,120);

<b> Paso seis (Opcional) : </b> Vamos a definir el estilo del histograma. Se va a definir el color de la linea y el relleno de los histogramas que se crearon en la celda anterior. Como referencia para los colores permitidos por ROOT se tiene la siguiente paleta, y para mas informacion se puede acceder al siguiente enlace <b><a href="#"> https://root.cern/doc/v610/classTColor.html</a> :
  
  <div id="Z_analysis" style="float:left; clear:both; padding: 0.0cm 0.0cm 0.0cm 0.0cm; text-align: center; width:92%; margin:0px auto; background-color:#d1ecfc; border:none!important;">
          <img class="img-responsive" src="https://root.cern/doc/v610/pict1_TColor_002.png" alt="Visualisations" style="width:35%">
  </div> 

In [ ]:
h_Inv_Mass->SetFillColor(kBlue);
h_Inv_Mass->SetLineColor(kBlue);
h_Inv_Mass->SetMarkerSize(2.0);

<b> Paso Opcional </b>: Las siguientes dos lineas son un mecanismo para tomar solamente una fraccion de los eventos sobre los cuales se quiere correr el analisis, con el fin de minimizar el tiempo que le lleva correr el loop.


In [ ]:
int nentries, nbytes, i;
nentries = (Int_t)MonteCarlo->GetEntries();

In [ ]:
// IMPORTANT: fraction events we want to run
fraction_events = 0.001;
events_to_run = nentries*fraction_events;

std::cout << "Total # events = "  << nentries
          << ". Events to run = " << events_to_run
          << " corresponding to " << fraction_events*100
          << "% of total events!" << std::endl;

<b> Paso Siete: </b> Aqui vamos a hacer el loop. Se va a realizar una serie de cortes para poder seleccionar eventos que cumplan solamente ciertos criterios, tal que coincida con lo esperado para este analisis, y llenar los histogramas definidos en el paso cinco solamente con los eventos que pasen la seleccion. Los criterios son los siguientes:
    <ol>
        <li> Deben haber al menos 2 buenos leptones </li>
        <li> Los leptones seleccionados deben ser de la misma familia </li>
        <li> Los leptones seleccionados deben tener carga opuesta </li>
    </ol>
    
 La masa invariante puede ser calculada como la sumatoria del modulo del cuadri-momentum de las particulas en las cuales decae la particula inicial de la siguiente forma:
 
$M^2 = (\sum E)^2   - ||\sum p ||^2 = (E_1 + E_2)^2 - (p_{x1}^2 + p_{x2}^2)^2 - (p_{y1}^2 + p_{y2}^2)^2 - (p_{z1}^2 + p_{z2}^2)^2 $, 

Nosotros lo vamos a hacer por medio de la clase de TLorentzVector, la cual se puede encontrar informacion en: <b><a href="#">https://root.cern.ch/doc/master/classTLorentzVector.html</a> :

In [ ]:

TLorentzVector l1;         
TLorentzVector l2;  
TLorentzVector l1plusl2;

for (i=0; i<events_to_run; i++)
{
    nbytes = MonteCarlo->GetEntry(i);
    
    // Corte uno: El numero de leptones en los eventos debe ser exactamente 2
    if(MonteCarlo_lepton_n==2) 
    {
        
        //Corte Dos: Leptones de la misma familia
        if(MonteCarlo_lepton_type[0] == MonteCarlo_lepton_type[1]) 
        {
            // Corte Tres: Los dos leptones seleccionados deben tener carga opuesta 
            if(MonteCarlo_lepton_charge[0] != MonteCarlo_lepton_charge[1])  {
                l1.SetPtEtaPhiE(MonteCarlo_lepton_pt[0]/1000., MonteCarlo_lepton_eta[0], MonteCarlo_lepton_phi[0], MonteCarlo_lepton_E[0]/1000.);
                l2.SetPtEtaPhiE(MonteCarlo_lepton_pt[1]/1000., MonteCarlo_lepton_eta[1], MonteCarlo_lepton_phi[1], MonteCarlo_lepton_E[1]/1000.);
                l1plusl2=l1+l2;
                //l1plusl2.M();
                h_Inv_Mass->Fill(l1plusl2.M());

            }
        }
    }
}

<b> Paso Ocho: </b> Por ultimo vamos a dibujar los histogramas que hemos creado y llenado en los pasos anteriornes. Para ello primero es necesario crear una canvas, esta es el area en el la cual va a permitir que se puedan ver los histogramas dibujados. 

In [ ]:
TCanvas *cz = new TCanvas("cz","cz",10,10,700,500);
h_Inv_Mass->Draw();
cz->Draw();

# Ahora repitamos lo anterior para los Datos Reales!

In [ ]:
TChain *data = new TChain("mini");

In [ ]:
data->Add("/media/sf_ATLAS_OpenData/Data/DataEgamma.root");
data->Add("/media/sf_ATLAS_OpenData/Data/DataMuons.root");


In [ ]:
const int vd = 5;

Int_t   data_lepton_n = -1,
        data_runNumber = -1,  
        data_lepton_charge[vd], 
        data_lepton_type[vd];

Float_t data_lepton_pt[vd],
        data_lepton_eta[vd],
        data_lepton_phi[vd],
        data_lepton_E[vd];
        
Float_t data_met_et = -1,
        data_met_phi = -1;


In [ ]:
data->SetBranchAddress("lep_n",      &data_lepton_n);
data->SetBranchAddress("channelNumber",  &data_runNumber);
data->SetBranchAddress("lep_charge", &data_lepton_charge);
data->SetBranchAddress("lep_type",   &data_lepton_type);
data->SetBranchAddress("lep_pt",     &data_lepton_pt);
data->SetBranchAddress("lep_eta",    &data_lepton_eta);
data->SetBranchAddress("lep_phi",    &data_lepton_phi);
data->SetBranchAddress("lep_E",   &data_lepton_E);

data->SetBranchAddress("met_et",   &data_met_et);
data->SetBranchAddress("met_phi",   &data_met_phi);


In [ ]:
TH1F *h_Inv_Mass_data = new TH1F("h_Inv_Mass_data","Masa Invariante Candidato a Z; Masa Invariante [GeV]; eventos",20,60,120);


In [ ]:
h_Inv_Mass_data->SetFillColor(kAzure);
h_Inv_Mass_data->SetLineColor(kAzure);
h_Inv_Mass_data->SetMarkerSize(1.0);

In [ ]:
int mentries, mbytes, j;
mentries = (Int_t)data->GetEntries();

In [ ]:
// IMPORTANT: fraction events we want to run
fraction_events = 0.001;
events_to_run = mentries*fraction_events;

std::cout << "Total # events = "  << mentries
          << ". Events to run = " << events_to_run
          << " corresponding to " << fraction_events*100
          << "% of total events!" << std::endl;

In [ ]:

TLorentzVector Lepton1;         
TLorentzVector Lepton2;  
TLorentzVector Lepton1plusLepton2;

for (i=0; i<events_to_run; i++)
{
    mbytes = data->GetEntry(i);
    
    // Corte uno: El numero de leptones en los eventos debe ser exactamente 2
    if(data_lepton_n==2) 
    {
        //Corte Dos: Leptones de la misma familia
        if(data_lepton_type[0] == data_lepton_type[1]) 
        {
            // Corte Tres: Los dos leptones seleccionados deben tener carga opuesta 
            if(data_lepton_charge[0] != data_lepton_charge[1]) 
            {
                Lepton1.SetPtEtaPhiE(data_lepton_pt[0]/1000., data_lepton_eta[0], data_lepton_phi[0], data_lepton_E[0]/1000.);
                Lepton2.SetPtEtaPhiE(data_lepton_pt[1]/1000., data_lepton_eta[1], data_lepton_phi[1], data_lepton_E[1]/1000.);
                Lepton1plusLepton2=Lepton1+Lepton2;
                //l1plusl2.M();
                h_Inv_Mass_data->Fill(Lepton1plusLepton2.M());

            }
        }
    }
}

In [ ]:
TCanvas *cz1 = new TCanvas("cz1","cz1",10,10,700,500);
h_Inv_Mass_data->Draw();
cz1->Draw();

In [ ]:
TCanvas *cz2 = new TCanvas("cz2","cz2",10,10,700,500);

h_Inv_Mass->Draw();
h_Inv_Mass_data->Draw("same");

cz2->Draw();